<a href="https://colab.research.google.com/github/jo-cho/advances_in_financial_machine_learning/blob/master/data_structures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

In [3]:
!pip install -q mlfinlab
from mlfinlab import data_structures, features, filters, labeling, util

     |████████████████████████████████| 122kB 9.9MB/s 


In [4]:
# 데이터 (Chapter2)

raw_dollar_bars = data_structures.get_dollar_bars('/content/drive/My Drive/Colab Notebooks/csv/clean_IVE_tickbidask2.csv', threshold=1000000)
dollar_bars = raw_dollar_bars.set_index(pd.to_datetime(raw_dollar_bars.date_time))
dollar_bars = dollar_bars.drop(columns='date_time')
dollar_bars = dollar_bars.reset_index().drop_duplicates(subset='date_time', keep='last').set_index('date_time')

Reading data in batches:
Batch number: 0
Returning bars 



y

In [0]:
# triple-barrier (Chapter3)

close = dollar_bars.close
pt_sl = [1,1]
target = util.get_daily_vol(close)
h = target.mean()
t_events = filters.cusum_filter(close,h)
vertical_barrier_times = labeling.add_vertical_barrier(t_events,close,num_days=1)

In [0]:
# side (Chapter3)

fast_window = 20
slow_window = 50

df = pd.DataFrame()
df['fast_mavg'] = close.rolling(window=fast_window, min_periods=fast_window, center=False).mean()
df['slow_mavg'] = close.rolling(window=slow_window, min_periods=slow_window, center=False).mean()
df['side'] = np.nan

long_signals = df['fast_mavg'] >= df['slow_mavg'] 
short_signals = df['fast_mavg'] < df['slow_mavg'] 
df.loc[long_signals, 'side'] = 1
df.loc[short_signals, 'side'] = -1

df['side'] = df['side'].shift(1)

side = df['side']

In [0]:
# bin without side
events = labeling.get_events(close, t_events, pt_sl, target, min_ret=0.005, num_threads=1, vertical_barrier_times = vertical_barrier_times)
bin = labeling.get_bin(events, close)

In [7]:
# bin with side (multi-label)

events = labeling.get_events(close, t_events, pt_sl, target, min_ret=0.005, num_threads=1, vertical_barrier_times = vertical_barrier_times ,side_prediction = side)
bin = labeling.get_bins(events,close)

/usr/local/lib/python3.6/dist-packages/mlfinlab/labeling/labeling.py:124: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  target = target.loc[t_events]


In [15]:
print(bin.bin)
# this is label y

2009-09-30 09:44:47    0
2009-09-30 09:56:31    0
2009-09-30 12:48:14    0
2009-09-30 15:05:38    1
2009-10-01 09:52:26    1
                      ..
2019-06-12 10:18:41    0
2019-06-17 09:57:13    1
2019-06-18 09:39:43    0
2019-06-20 09:30:00    0
2019-06-20 12:20:48    1
Name: bin, Length: 2665, dtype: int64


In [0]:
y= bin[['bin']]

X (features)

In [0]:
raw_data = pd.DataFrame()

# Log Returns
raw_data['log_ret'] = np.log(close).diff()

# Momentum
raw_data['mom1'] = close.pct_change(periods=1)
raw_data['mom2'] = close.pct_change(periods=2)
raw_data['mom3'] = close.pct_change(periods=3)
raw_data['mom4'] = close.pct_change(periods=4)
raw_data['mom5'] = close.pct_change(periods=5)

# Volatility
raw_data['volatility_50'] = raw_data['log_ret'].rolling(window=50, min_periods=50, center=False).std()
raw_data['volatility_31'] = raw_data['log_ret'].rolling(window=31, min_periods=31, center=False).std()
raw_data['volatility_15'] = raw_data['log_ret'].rolling(window=15, min_periods=15, center=False).std()

# Serial Correlation (Takes about 4 minutes)
window_autocorr = 50

raw_data['autocorr_1'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=1), raw=False)
raw_data['autocorr_2'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=2), raw=False)
raw_data['autocorr_3'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=3), raw=False)
raw_data['autocorr_4'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=4), raw=False)
raw_data['autocorr_5'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=5), raw=False)

# Get the various log -t returns
raw_data['log_t1'] = raw_data['log_ret'].shift(1)
raw_data['log_t2'] = raw_data['log_ret'].shift(2)
raw_data['log_t3'] = raw_data['log_ret'].shift(3)
raw_data['log_t4'] = raw_data['log_ret'].shift(4)
raw_data['log_t5'] = raw_data['log_ret'].shift(5)

In [0]:
raw_data = raw_data.shift(1)

In [0]:
raw_data['side'] = side

In [32]:
raw_data2 = raw_data.dropna()
X = raw_data2.loc[y.index,:]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [36]:
data = X.join(y,how='right').dropna()
data.head()

,log_ret,mom1,mom2,mom3,mom4,mom5,volatility_50,volatility_31,volatility_15,autocorr_1,autocorr_2,autocorr_3,autocorr_4,autocorr_5,log_t1,log_t2,log_t3,log_t4,log_t5,side,bin
2009-10-01 09:52:26,-0.002947,-0.002942,-0.006450,-0.006062,-0.004699,-0.010319,0.002759,0.003416,0.003565,-0.048314,0.173996,-0.255830,0.050401,-0.127175,-0.003525,0.000391,0.001370,-0.005662,-0.000710,-1.0,1
2009-10-01 11:16:04,0.002571,0.002575,0.002358,-0.000395,-0.000395,0.000989,0.002866,0.002943,0.002367,-0.075463,0.143150,-0.197550,0.057683,-0.151334,-0.000216,-0.002751,0.000000,0.001383,-0.004538,-1.0,1
2009-10-01 15:51:53,0.000200,0.000200,0.000148,-0.000997,-0.001774,-0.003184,0.002846,0.002476,0.002253,-0.124987,0.026404,-0.106176,0.001387,-0.147859,-0.000052,-0.001145,-0.000778,-0.001414,0.001394,-1.0,1
2009-10-01 16:16:36,-0.002206,-0.002203,-0.005390,-0.005192,-0.005243,-0.006382,0.002730,0.002351,0.002189,-0.003966,-0.059657,-0.076941,-0.003204,-0.125404,-0.003199,0.000200,-0.000052,-0.001145,-0.000778,-1.0,1
2009-10-02 09:32:42,0.009919,0.009968,0.007743,0.004524,0.004724,0.004672,0.003018,0.002958,0.003157,-0.011765,-0.137563,-0.046403,-0.006264,-0.085668,-0.002206,-0.003199,0.000200,-0.000052,-0.001145,-1.0,0
